<a href="https://colab.research.google.com/github/saraMuay/Topic-Analysis-of-Review-Data/blob/main/Topic_Analysis_of_Review_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyLDAvis
!pip install NLTK

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import re
import string
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gan
import matplotlib.pyplot as plt

import warnings 

warnings.filterwarnings('ignore') 
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
nltk.download('all')

Read the .csv file using Pandas. Take a look at the top few records.

In [4]:
!git clone https://github.com/saraMuay/Topic-Analysis-of-Review-Data.git


Cloning into 'Topic-Analysis-of-Review-Data'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [5]:
Data = pd.read_csv('https://raw.githubusercontent.com/saraMuay/Topic-Analysis-of-Review-Data/main/K8%20Reviews%20v0.2.csv' )
print(Data.head())

   sentiment                                             review
0          1             Good but need updates and improvements
1          0  Worst mobile i have bought ever, Battery is dr...
2          1  when I will get my 10% cash back.... its alrea...
3          1                                               Good
4          0  The worst phone everThey have changed the last...


Normalize casings for the review text and extract the text into a list for easier manipulation.

In [6]:
def NormalizeCasings(review):
    Reviews = []
    for i in review:
      Reviews.append(i.lower())
        
    return Reviews

In [32]:
Normalized = NormalizeCasings(Data['review'])
Normalized[0:10]

['good but need updates and improvements',
 "worst mobile i have bought ever, battery is draining like hell, backup is only 6 to 7 hours with internet uses, even if i put mobile idle its getting discharged.this is biggest lie from amazon & lenove which is not at all expected, they are making full by saying that battery is 4000mah & booster charger is fake, it takes at least 4 to 5 hours to be fully charged.don't know how lenovo will survive by making full of us.please don;t go for this else you will regret like me.",
 'when i will get my 10% cash back.... its already 15 january..',
 'good',
 'the worst phone everthey have changed the last phone but the problem is still same and the amazon is not returning the phone .highly disappointing of amazon',
 "only i'm telling don't buyi'm totally disappointedpoor batterypoor camerawaste of money",
 'phone is awesome. but while charging, it heats up allot..really a genuine reason to hate lenovo k8 note',
 'the battery level has worn down',
 "it'

- Tokenize the reviews using NLTKs word_tokenize function.

- Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.

- For the topic model, we should  want to include only nouns.

   - Find out all the POS tags that correspond to nouns.
   - Limit the data to only terms with these tags.

In [8]:
def Tokenize(review):
    Reviews = []
    for i in range(len(review)):
      TokenToWord = nltk.word_tokenize(review[i])
      TagPos = nltk.pos_tag(TokenToWord)

      for word, pos in TagPos:
        if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
                Reviews.append(review[i])   
    return Reviews 


In [33]:
Tokenizing = Tokenize(Normalized)
Tokenizing[0:10]

['good but need updates and improvements',
 'good but need updates and improvements',
 "worst mobile i have bought ever, battery is draining like hell, backup is only 6 to 7 hours with internet uses, even if i put mobile idle its getting discharged.this is biggest lie from amazon & lenove which is not at all expected, they are making full by saying that battery is 4000mah & booster charger is fake, it takes at least 4 to 5 hours to be fully charged.don't know how lenovo will survive by making full of us.please don;t go for this else you will regret like me.",
 "worst mobile i have bought ever, battery is draining like hell, backup is only 6 to 7 hours with internet uses, even if i put mobile idle its getting discharged.this is biggest lie from amazon & lenove which is not at all expected, they are making full by saying that battery is 4000mah & booster charger is fake, it takes at least 4 to 5 hours to be fully charged.don't know how lenovo will survive by making full of us.please don;

In [34]:
UniqeTokenizing = [] 
for i in Tokenizing:
  if i not in UniqeTokenizing:
      UniqeTokenizing.append(i)

print ( 'Tokenizing', len(Tokenizing))
print ( 'UniqeTokenizing', len(UniqeTokenizing))
UniqeTokenizing[0:10]

Tokenizing 90723
UniqeTokenizing 12384


['good but need updates and improvements',
 "worst mobile i have bought ever, battery is draining like hell, backup is only 6 to 7 hours with internet uses, even if i put mobile idle its getting discharged.this is biggest lie from amazon & lenove which is not at all expected, they are making full by saying that battery is 4000mah & booster charger is fake, it takes at least 4 to 5 hours to be fully charged.don't know how lenovo will survive by making full of us.please don;t go for this else you will regret like me.",
 'when i will get my 10% cash back.... its already 15 january..',
 'the worst phone everthey have changed the last phone but the problem is still same and the amazon is not returning the phone .highly disappointing of amazon',
 "only i'm telling don't buyi'm totally disappointedpoor batterypoor camerawaste of money",
 'phone is awesome. but while charging, it heats up allot..really a genuine reason to hate lenovo k8 note',
 'the battery level has worn down',
 "it's over hi

 Remove stopwords and punctuation

In [12]:
for i in range (len(UniqeTokenizing)):
  UniqeTokenizing[i] = re.sub('[%s]' % re.escape(string.punctuation), '',UniqeTokenizing[i])   # Removing punctuations marks
  lemmatizer = WordNetLemmatizer()
  UniqeTokenizing[i]= lemmatizer.lemmatize(UniqeTokenizing[i]) 

In [13]:
def Remove_Stopwords(review):
  for i in range(len(review)):
    words = nltk.word_tokenize(review[i])                                 
    words = [x for x in words if x not in stopwords.words('english')] 
    review[i] = ' '.join(words)
    
    return review

In [35]:
PreProcessReviews = Remove_Stopwords (UniqeTokenizing)
PreProcessReviews[0:10]

['good need updates improvements',
 "worst mobile i have bought ever, battery is draining like hell, backup is only 6 to 7 hours with internet uses, even if i put mobile idle its getting discharged.this is biggest lie from amazon & lenove which is not at all expected, they are making full by saying that battery is 4000mah & booster charger is fake, it takes at least 4 to 5 hours to be fully charged.don't know how lenovo will survive by making full of us.please don;t go for this else you will regret like me.",
 'when i will get my 10% cash back.... its already 15 january..',
 'the worst phone everthey have changed the last phone but the problem is still same and the amazon is not returning the phone .highly disappointing of amazon',
 "only i'm telling don't buyi'm totally disappointedpoor batterypoor camerawaste of money",
 'phone is awesome. but while charging, it heats up allot..really a genuine reason to hate lenovo k8 note',
 'the battery level has worn down',
 "it's over hitting pr

Lemmatize. 

In [16]:
  lemmatizer = WordNetLemmatizer()
  for i in range (len(PreProcessReviews)):
    PreProcessReviews[i] = lemmatizer.lemmatize(PreProcessReviews[i])

In [36]:
PreProcessReviews[0:10]

['good need updates improvements',
 "worst mobile i have bought ever, battery is draining like hell, backup is only 6 to 7 hours with internet uses, even if i put mobile idle its getting discharged.this is biggest lie from amazon & lenove which is not at all expected, they are making full by saying that battery is 4000mah & booster charger is fake, it takes at least 4 to 5 hours to be fully charged.don't know how lenovo will survive by making full of us.please don;t go for this else you will regret like me.",
 'when i will get my 10% cash back.... its already 15 january..',
 'the worst phone everthey have changed the last phone but the problem is still same and the amazon is not returning the phone .highly disappointing of amazon',
 "only i'm telling don't buyi'm totally disappointedpoor batterypoor camerawaste of money",
 'phone is awesome. but while charging, it heats up allot..really a genuine reason to hate lenovo k8 note',
 'the battery level has worn down',
 "it's over hitting pr

Create a topic model using LDA on the cleaned-up data with 12 topics

In [18]:
TokenizeReviews = []
for i in PreProcessReviews:
    TokenizeReviews.append(nltk.word_tokenize(i)) 

WordID = corpora.Dictionary(TokenizeReviews)
corpus = [WordID.doc2bow(text) for text in TokenizeReviews]


In [22]:
print('corpus : ',corpus[:1])
print('WordID : ',WordID[0])

CountFreq = [[(WordID[id], freq) for id, freq in cp] for cp in corpus]
CountFreq[:1]

corpus :  [[(0, 1), (1, 1), (2, 1), (3, 1)]]
WordID :  good


[[('good', 1), ('improvements', 1), ('need', 1), ('updates', 1)]]

In [23]:
LDA_Model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=WordID,num_topics=12,random_state=100,
                                           update_every=1,chunksize=100,passes=10,alpha='auto',per_word_topics=True)

In [24]:
# Print out the top terms for each topic.
pprint(LDA_Model.print_topics())

[(0,
  '0.114*"lenovo" + 0.080*"note" + 0.064*"k8" + 0.029*"which" + 0.023*"do" + '
  '0.020*"off" + 0.018*"phones" + 0.016*"heats" + 0.015*"dolby" + '
  '0.015*"feature"'),
 (1,
  '0.041*"got" + 0.041*"excellent" + 0.029*"speed" + 0.026*"should" + '
  '0.022*"experience" + 0.020*"video" + 0.019*"found" + 0.017*"till" + '
  '0.017*"again" + 0.017*"down"'),
 (2,
  '0.139*"battery" + 0.049*"charging" + 0.046*"nice" + 0.033*"fast" + '
  '0.029*"by" + 0.024*"getting" + 0.019*"4" + 0.017*"full" + 0.016*"work" + '
  '0.015*"hours"'),
 (3,
  '0.126*"mobile" + 0.101*"product" + 0.069*"buy" + 0.056*"dont" + '
  '0.050*"worst" + 0.050*"bad" + 0.046*"amazon" + 0.039*"awesome" + '
  '0.016*"delivery" + 0.015*"ever"'),
 (4,
  '0.070*"the" + 0.059*"and" + 0.051*"to" + 0.042*"it" + 0.042*"this" + '
  '0.036*"for" + 0.035*"a" + 0.030*"of" + 0.028*"in" + 0.027*"with"'),
 (5,
  '0.181*"is" + 0.092*"phone" + 0.084*"not" + 0.083*"good" + 0.058*"very" + '
  '0.056*"camera" + 0.038*"but" + 0.024*"quality" +

In [25]:
doc = LDA_Model[corpus]

In [26]:
#coherence of the model with the c_v metric 

print('Model performance by Perplexity: ', np.around(LDA_Model.log_perplexity(corpus),2))  

Coherence_LDA_Model = CoherenceModel(model=LDA_Model, texts=TokenizeReviews, dictionary=WordID, coherence='c_v')
Coherence_LDA = Coherence_LDA_Model.get_coherence()
print('Coherence Score: ', np.around(Coherence_LDA,2))

Model performance by Perplexity:  -7.35
Coherence Score:  0.39


In [27]:
train_vecs = []
for i in range(len(TokenizeReviews)):
    top_topics = LDA_Model.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(12)]
    train_vecs.append(topic_vec)

In [28]:
train_vecs[0]

[0.041749276,
 0.026413478,
 0.066789,
 0.057074215,
 0.3104785,
 0.24485344,
 0.01939744,
 0.06028511,
 0.0252962,
 0.06437967,
 0.033413175,
 0.04987044]

Create a topic model using LDA with what you think is the optimal number of topics

In [29]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=WordID,num_topics = 7,random_state=100,update_every=1,
                                           chunksize=100,passes=10,alpha='auto',per_word_topics=True)

In [30]:
#Print out the top terms for each topic.

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.098*"i" + 0.041*"lenovo" + 0.029*"note" + 0.029*"my" + 0.024*"was" + '
  '0.023*"k8" + 0.022*"from" + 0.019*"amazon" + 0.015*"am" + 0.015*"will"'),
 (1,
  '0.041*"low" + 0.035*"excellent" + 0.023*"should" + 0.021*"light" + '
  '0.018*"fine" + 0.015*"till" + 0.014*"down" + 0.013*"expected" + '
  '0.011*"side" + 0.011*"notification"'),
 (2,
  '0.066*"battery" + 0.057*"mobile" + 0.031*"problem" + 0.031*"heating" + '
  '0.027*"charging" + 0.025*"nice" + 0.022*"up" + 0.021*"issue" + 0.018*"fast" '
  '+ 0.018*"time"'),
 (3,
  '0.063*"best" + 0.052*"awesome" + 0.047*"by" + 0.036*"days" + '
  '0.022*"delivery" + 0.018*"out" + 0.017*"amazing" + 0.016*"satisfied" + '
  '0.015*"missing" + 0.011*"charged"'),
 (4,
  '0.051*"phone" + 0.049*"the" + 0.041*"and" + 0.035*"this" + 0.033*"to" + '
  '0.029*"in" + 0.029*"it" + 0.025*"for" + 0.025*"a" + 0.021*"of"'),
 (5,
  '0.180*"is" + 0.102*"not" + 0.087*"good" + 0.060*"very" + 0.058*"camera" + '
  '0.033*"product" + 0.025*"quality" + 0.024*"but

In [31]:
# coherence of the model with the c_v metric 

print('Model performance by Perplexity: ',np.around( lda_model.log_perplexity(corpus),2))  

coherence_model_lda = CoherenceModel(model=lda_model, texts=TokenizeReviews, dictionary=WordID, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', np.around(coherence_lda,2))

Model performance by Perplexity:  -7.1
Coherence Score:  0.44
